### Some handy util tools

each cell should provide different functions

In [ ]:
## Copy all training images specified in split.csv to another folder
import pandas as pd
import shutil 
SPLIT_CSV          = 'interim/by_dop80c_1312.split.csv'
DEST_TRAINING_PATH = 'interim/by_dop80c_1312'
split_df = pd.read_csv(SPLIT_CSV)
for index, d in split_df[split_df['train']].iterrows():
  #print (d.img_filepath)
  shutil.copy(d.img_filepath, DEST_TRAINING_PATH)

In [ ]:
# copy files
import shutil 
from pathlib import Path 
import os

ds = 'by_dop80c_1312' # 'opendata_luftbild_dop60_1312'
FROM_DIR = Path('aerial_images_resampled/{ds}'.format(ds=ds))
DST_DIR = Path('interim/{ds}/deepforest_r1/train2'.format(ds=ds))
PATTERN = [
 '1288638.722245550_6129295.399946138_1289422.201785473_6130078.879486061.tiff',
 '1288638.722245550_6128569.248177430_1289422.201785473_6129352.727717352.tiff',
 '1290091.025782968_6128569.248177430_1290874.505322890_6129352.727717352.tiff',
 '1290817.177551676_6127843.096408719_1291600.657091599_6128626.575948643.tiff',
 '1287186.418708133_6127116.944640011_1287969.898248056_6127900.424179933.tiff',
 '1290091.025782968_6127843.096408719_1290874.505322890_6128626.575948643.tiff',
 '1289364.874014259_6128569.248177430_1290148.353554182_6129352.727717352.tiff',
 '1287912.570476842_6129295.399946138_1288696.050016765_6130078.879486061.tiff',
 '1288638.722245550_6130021.551714847_1289422.201785473_6130805.031254769.tiff']
os.makedirs(DST_DIR, exist_ok=True)
for pattern in PATTERN:
  for s in FROM_DIR.glob(pattern):
    print(s)
    shutil.copy(s, DST_DIR)


In [ ]:
## Downsampling map tiles from a higher zoom level down to specified lower zoom levels 
# 
import mercantile
import supermercado
import rasterio
from rasterio import plot, transform
import numpy
import json
import os
import pathlib

AREA = 'munich.boundary.geojson'
SRC_ZOOM = 11
SOURCE_MAP_TILE_BAND_COUNT = 4
SOURCE_MAP_TILE_WIDTH_PX   = 256 
SOURCE_MAP_TILE_HEIGHT_PX  = 256
SOURCE_MAP_TILE_DTYPE      = numpy.uint8
SOURCE_MAP_TILE_TYPE       = ".tiff"
OUTPUT_ZOOMS               = range(0, SRC_ZOOM)
SOURCE_MAP_TILE_PATH       = 'aerial_images/opendata_luftbild_dop60_2017'
OUTPUT_TILE_PATH           = 'aerial_images/opendata_luftbild_dop60_2017'
SOURCE_MAP_TILE_EPSG       = 3857 # only epsg:3857 is supported

f = open(AREA)
area = json.load(f)
f.close()

features = area["features"]
features = [f for f in supermercado.super_utils.filter_features(features)]
for z in reversed(OUTPUT_ZOOMS):
  for t in supermercado.burntiles.burn(features, z):
    tile = t.tolist()
    #print(tile)
    children = mercantile.children(tile)
    
    temp = numpy.zeros((SOURCE_MAP_TILE_BAND_COUNT, SOURCE_MAP_TILE_HEIGHT_PX*2, SOURCE_MAP_TILE_WIDTH_PX*2), dtype=SOURCE_MAP_TILE_DTYPE)

    for y in range(children[1].y, children[3].y+1):
      for x in range(children[0].x, children[1].x+1):
        try:
          path = SOURCE_MAP_TILE_PATH if children[0].z == SRC_ZOOM else OUTPUT_TILE_PATH
          child = path + "/" + str(children[0].z) + "/" + str(x) + "/" + str(y) + SOURCE_MAP_TILE_TYPE
          with rasterio.open(child) as tile_src:
            data_src = tile_src.read()            
            temp[:, 
                (y-children[1].y)*SOURCE_MAP_TILE_HEIGHT_PX:(y-children[1].y+1)*SOURCE_MAP_TILE_HEIGHT_PX, 
                (x-children[0].x)*SOURCE_MAP_TILE_WIDTH_PX :(x-children[0].x+1)*SOURCE_MAP_TILE_WIDTH_PX] = data_src[:,:,:]
        except rasterio.errors.RasterioIOError as e:
            pass
    dest_path = OUTPUT_TILE_PATH + "/" + str(tile[2]) + "/" + str(tile[0]) + "/" + str(tile[1]) + SOURCE_MAP_TILE_TYPE
    os.makedirs(pathlib.Path(dest_path).parent, exist_ok=True)
    bb = mercantile.xy_bounds(tile[0], tile[1], tile[2])
    tile_tf = rasterio.transform.from_bounds(bb.left, bb.bottom, bb.right, bb.top, SOURCE_MAP_TILE_WIDTH_PX, SOURCE_MAP_TILE_HEIGHT_PX)
    with rasterio.open(dest_path, 'w', driver='GTiff',
                width=SOURCE_MAP_TILE_WIDTH_PX, height=SOURCE_MAP_TILE_HEIGHT_PX,
                count=SOURCE_MAP_TILE_BAND_COUNT, dtype=SOURCE_MAP_TILE_DTYPE, nodata=0,
                transform=tile_tf, 
                crs=rasterio.crs.CRS.from_epsg(SOURCE_MAP_TILE_EPSG)) as dst:
      dst.write(temp)
    #rasterio.plot.show(temp)

In [ ]:
## convert tiff to png
#
from pathlib import Path
import os

os.environ['GDAL_PAM_ENABLED'] = 'NO'

PATH = 'aerial_images/opendata_luftbild_dop60_2017_wip/'

for path in Path(PATH).rglob('*.tiff'):
    with rasterio.open(path) as src:
      dest_path = path.parent.joinpath(path.stem+'.png')
      with rasterio.open(dest_path, 'w',
                         driver='PNG',
                         height=src.shape[0],
                         width=src.shape[1],
                         count=src.count,
                         dtype=src.meta['dtype'],
                         nodata=0,
                         compress='deflate') as dst:
        dst.write(src.read())

#for path in Path(PATH).rglob('*.aux.xml'):
#  os.remove(path)

for path in Path(PATH).rglob('*.tiff'):
  os.remove(path)
    

In [ ]:
## show image and meta info
#
import rasterio
from rasterio import plot

PATH = 'temp/png/12/2177/1420.png'

with rasterio.open(PATH) as src:
  print(src.meta)
  rasterio.plot.show(src.read())

In [ ]:
import math
# resolution of a map tile
lat = 48.137154
z = 18
resolution =  -156543.04 * math.cos(lat) / (2**z)
resolution # meter per pixel

In [25]:
# remove "small trees" labels in labelme annotation json files

import glob
import math
import json
import pathlib
from pathlib import Path

ds = 'by_dop80c_1312' # 'opendata_luftbild_dop60_1312' #

FILTER_LABEL = "Tree"
FILTER_TYPE  = "circle"
FILTER_DIAMETER = 10 # pixel
labels = glob.glob('interim/{ds}/deepforest_r1/train2/*.json'.format(ds=ds))

def diameter(points):
  p1 = points[0]
  p2 = points[1]
  return 2 * math.sqrt(math.pow(p1[0]-p2[0],2) + math.pow(p1[1]-p2[1],2))

def filter(shape):
  if shape['label'] == FILTER_LABEL and \
     shape['shape_type'] == FILTER_TYPE and \
     diameter(shape['points']) >= FILTER_DIAMETER:
    return True
  return False

for label in labels:
  f = Path(label)
  gjson = None
  print(f)
  with open(f) as json_file:
    gjson = json.load(json_file)
  gjson['shapes'] = [s for s in gjson['shapes'] if filter(s)]
  with open(f, 'w') as outfile:
    json.dump(gjson, outfile, indent=2)

interim\by_dop80c_1312\deepforest_r1\train2\1287186.418708133_6127116.944640011_1287969.898248056_6127900.424179933.json
interim\by_dop80c_1312\deepforest_r1\train2\1287912.570476842_6129295.399946138_1288696.050016765_6130078.879486061.json
interim\by_dop80c_1312\deepforest_r1\train2\1288638.722245550_6128569.248177430_1289422.201785473_6129352.727717352.json
interim\by_dop80c_1312\deepforest_r1\train2\1288638.722245550_6129295.399946138_1289422.201785473_6130078.879486061.json
interim\by_dop80c_1312\deepforest_r1\train2\1288638.722245550_6130021.551714847_1289422.201785473_6130805.031254769.json
interim\by_dop80c_1312\deepforest_r1\train2\1289364.874014259_6128569.248177430_1290148.353554182_6129352.727717352.json
interim\by_dop80c_1312\deepforest_r1\train2\1290091.025782968_6127843.096408719_1290874.505322890_6128626.575948643.json
interim\by_dop80c_1312\deepforest_r1\train2\1290091.025782968_6128569.248177430_1290874.505322890_6129352.727717352.json
interim\by_dop80c_1312\deepfores

In [ ]:
# recreate labeme annotation json file from (e.g. deepforest) annotation csv

import json
import csv
import os
import pandas as pd
import glob
from pathlib import Path

PATH = "interim/by_dop80c_1312/deepforest_r1/response/crop/"
imageHeight = imageWidth = 1312

for c in glob.glob(PATH + "*.csv_"):
  df = pd.read_csv(c)
  files = list(df['image_path'].unique())

  for file in files:
    label = { "version": "4.5.10",
              "flags": {},
              "shapes": [],
              "imagePath": Path(file).name,
              "imageData": None,
              "imageHeight": imageHeight,
              "imageWidth": imageWidth
            }
    bboxes = df[df['image_path'] == file]
    for index, row in bboxes.iterrows():
      shape = {
        "label": row["label"],
        "points": [
          [
            row["xmin"],
            row["ymin"]
          ],
          [
            row["xmax"],
            row["ymax"]
          ]
        ],
        "group_id": None,
        "shape_type": "rectangle",
        "flags": {}
      }
      label["shapes"].append(shape)

    dest = PATH + os.path.splitext(file)[0] + ".json"
    with open(dest, 'w') as outfile:
      json.dump(label, outfile, indent=2)

In [ ]:
# recreate labeme annotation json file from pickl

import json
import csv
import os
import pandas as pd
import glob
import rasterio
import torch
from torchvision.ops import nms
from pathlib import Path
import numpy as np

ds = 'opendata_luftbild_dop60_1312' #'by_dop80c_1312' # 
PICKL_DIR = Path('interim/{ds}/deepforest_r1/predict/b'.format(ds=ds))
IMG_DIR = Path('interim/{ds}/deepforest_r1/train2'.format(ds=ds))

def run_nms(df, iou_threshold):
  boxes = torch.tensor(df[["xmin", "ymin", "xmax", "ymax"]].values,
                          dtype=torch.float32)
  scores = torch.tensor(df.score.values, dtype=torch.float32)
  labels = df.label.values

  bbox_left_idx = nms(boxes=boxes, scores=scores, iou_threshold=iou_threshold).numpy()
  new_boxes = boxes[bbox_left_idx]
  new_labels = labels[bbox_left_idx]
  new_scores = scores[bbox_left_idx]
  image_detections = np.concatenate([
                new_boxes,
                np.expand_dims(new_labels, axis=1),
                np.expand_dims(new_scores, axis=1)
            ], axis=1)
  columns = {"xmin":"float64", "ymin":"float64", 
             "xmax":"float64", "ymax":"float64", 
             "label":"object", "score":"float64"}
  return pd.DataFrame(image_detections, columns=columns).astype(columns)
 
for f in IMG_DIR.glob('*.tiff'):
  with rasterio.open(f) as img:
    imageHeight = img.height
    imageWidth = img.width

  df = pd.read_pickle(PICKL_DIR.joinpath(f.stem + ".pkl"))
  df = run_nms(df, iou_threshold=0.1)

  label = { "version": "4.5.10",
              "flags": {},
              "shapes": [],
              "imagePath": f.name,
              "imageData": None,
              "imageHeight": imageHeight,
              "imageWidth": imageWidth
            }
  for index, row in df.iterrows():
    shape = {
        "label": row.label,
        "points": [
          [
            (row.xmin + row.xmax)/2.0,
            (row.ymin + row.ymax)/2.0
          ],
          [
            (row.xmin + row.xmax)/2.0,
            row.ymax
          ]
        ],
        "group_id": None,
        "shape_type": "circle",
        "flags": {}
      }
    label["shapes"].append(shape)
  dest = IMG_DIR.joinpath(f.stem +".json")
  with open(dest, 'w') as outfile:
    json.dump(label, outfile, indent=2)


In [ ]:
# create geojson

import pyproj
import json

SRC_IMG_PATH = "aerial_images_resampled/by_dop80c_1312"
BBOX_PATH = "interim/by_dop80c_1312/deepforest_r1/predict"

geoj = {
    "type": "MultiPolygon", 
    "coordinates": []
}

for f in SRC_IMG_PATH.glob('*.tiff'):
  with rasterio.open(f) as img:
    img_transform = img.transform
    img_crs = img.crs
  proj = pyproj.Transformer.from_crs(img_crs, 4326, always_xy=True)
  b = BBOX_PATH.joinpath(f.stem + ".pkl")
  df = pd.read_pickle(b)
  for _, r in df.iterrows():
    if (r.xmax - r.xmin > 10 and r.ymax - r.ymin > 10):
      tl = proj.transform(*(img_transform * [r.xmin, r.ymin]))
      bl = proj.transform(*(img_transform * [r.xmin, r.ymax]))
      br = proj.transform(*(img_transform * [r.xmax, r.ymax]))
      tr = proj.transform(*(img_transform * [r.xmax, r.ymin]))
      geoj['coordinates'].append([[list(tl) , list(tr), list(br), list(bl), list(tl)]])

with open("temp/bbox.geojson", 'w') as outfile:
      json.dump(geoj, outfile, indent=2)


In [1]:
import rasterio
from rasterio.windows import Window
from pyproj import Transformer
from pathlib import Path

TARGETS = [
    [1288564.72,6129835.21]
    #[1289077.52,6130669.92],
    #[1290666.87,6128809.29],
    #[1289457.09,6128794.88],
    #[1289140.41,6128725.97],
    #[1288642.94,6129806.16],
    #[1290563.23,6128527.89],
    #[1290889.08,6128257.02],
    #[1287439.46,6127456.51]
]
IMG_DIR = Path('aerial_images_resampled/opendata_luftbild_dop60_1312')

paths = set()
for p in IMG_DIR.glob("*.tiff"):
    with rasterio.open(p) as src:
        a = src.transform * [0, 0]
        b = src.transform * [src.width, src.height]
        for target in TARGETS: 
            if target[0] > a[0] and target[0] < b[0] and \
               target[1] < a[1] and target[1] > b[1]:
                paths.add(p.stem)
                break
print (list(paths))

['1287912.570476842_6129295.399946138_1288696.050016765_6130078.879486061']
